### Packages

In [1]:
import re
import math
import pandas as pd
import numpy as np

### Cville Open Data

In [2]:
tickets = pd.read_csv("parking_tickets_historical.csv")

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (2,6,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
tickets.shape

(435303, 20)

In [4]:
tickets.head()

,Index,RecordID,TicketNumber,DateIssued,StreetName,TimeIssued,StreetNumber,LicenseState,WaiverRequestDate,WaiverGrantedDate,AppealDate,AppealGrantedDate,ViolationDescription,AppealStatus,Location,LicensePlateAnon,newlocation,input_string,latitude,longitude
0,0,3001,185322,2016-04-27T11:45:05.000Z,7TH ST NE,11:45,400,VA,NaN,NaN,NaN,NaN,Overtime Parking,NaN,400 7TH ST NE,106713,"400 7TH ST NE Charlottesville, VA","400 7TH ST NE Charlottesville, VA",38.031652,-78.475925
1,1,3094,185414,2016-04-27T13:57:39.000Z,7TH ST NE,13:57,400,VA,NaN,NaN,NaN,NaN,Void,NaN,400 7TH ST NE,135871,"400 7TH ST NE Charlottesville, VA","400 7TH ST NE Charlottesville, VA",38.031652,-78.475925
2,2,3136,185462,2016-04-28T13:05:44.000Z,7TH ST NE,13:05,400,VA,NaN,NaN,NaN,NaN,Overtime Parking,NaN,400 7TH ST NE,110072,"400 7TH ST NE Charlottesville, VA","400 7TH ST NE Charlottesville, VA",38.031652,-78.475925
3,3,3137,185463,2016-04-28T13:06:43.000Z,7TH ST NE,13:06,400,VA,NaN,NaN,NaN,NaN,Overtime Parking,NaN,400 7TH ST NE,132625,"400 7TH ST NE Charlottesville, VA","400 7TH ST NE Charlottesville, VA",38.031652,-78.475925
4,4,3614,185970,2016-05-05T12:33:21.000Z,7TH ST NE,12:33,400,VA,NaN,NaN,NaN,NaN,Overtime Parking,NaN,400 7TH ST NE,71445,"400 7TH ST NE Charlottesville, VA","400 7TH ST NE Charlottesville, VA",38.031652,-78.475925


### Coding Datetime, City Block

In [5]:
from datetime import datetime
#date_string = tickets.DateIssued[0]
#datetime.strptime(date_string, "%Y-%m-%dT%H:%M:%S.%fZ")

datetime.datetime(2016, 4, 27, 11, 45, 5)

In [6]:
unique_ticket_locations = tickets.drop_duplicates(subset=['latitude', 'longitude'], keep='first')
unique_ticket_locations.dropna(subset=['StreetNumber'], inplace=True)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [7]:
def remove_alpha(street_num):
    return re.sub("[^0-9]", "", str(street_num))

def convert_to_block_number(street_num):
    try:
        block = int(math.floor(int(street_num) / 100.0)) * 100
        return int(block)
    except Exception as e:
        try:
            block = int(math.floor(int(remove_alpha(street_num)) / 100.0)) * 100
            return int(block)
        except Exception as e:
            print('Attempt Failed:', street_num, type(street_num), e, flush=True)
            return 9999

In [8]:
unique_ticket_locations['BlockNumber'] = unique_ticket_locations['StreetNumber'].apply(lambda x: convert_to_block_number(x))

Attempt Failed:            <class 'str'> invalid literal for int() with base 10: ''
Attempt Failed:            <class 'str'> invalid literal for int() with base 10: ''
Attempt Failed:            <class 'str'> invalid literal for int() with base 10: ''
Attempt Failed:            <class 'str'> invalid literal for int() with base 10: ''
Attempt Failed:            <class 'str'> invalid literal for int() with base 10: ''
Attempt Failed:            <class 'str'> invalid literal for int() with base 10: ''
Attempt Failed:            <class 'str'> invalid literal for int() with base 10: ''
Attempt Failed:            <class 'str'> invalid literal for int() with base 10: ''
Attempt Failed:            <class 'str'> invalid literal for int() with base 10: ''
Attempt Failed:            <class 'str'> invalid literal for int() with base 10: ''
Attempt Failed:            <class 'str'> invalid literal for int() with base 10: ''
Attempt Failed:            <class 'str'> invalid literal for int() with base

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [9]:
unique_ticket_locations_blocks = unique_ticket_locations[(unique_ticket_locations['BlockNumber'] < 5000) & (unique_ticket_locations['BlockNumber'] >= 100)]

In [10]:
unique_ticket_locations_blocks['DT'] = unique_ticket_locations['DateIssued'].apply(lambda x: datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [11]:
t = pd.date_range("00:00", "23:00", freq="60min").time

In [12]:
d = pd.date_range("01/01/18","12/31/18")

### Unique Blocks in the Data

In [13]:
b = unique_ticket_locations_blocks[['BlockNumber','StreetName']]
b.drop_duplicates(inplace = True)
b.shape

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


(3196, 2)

In [14]:
unique_ticket_locations_blocks['block'] = unique_ticket_locations_blocks["BlockNumber"].map(str) + " " + unique_ticket_locations_blocks["StreetName"]
unique_ticket_locations_blocks.head()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Index,RecordID,TicketNumber,DateIssued,StreetName,TimeIssued,StreetNumber,LicenseState,WaiverRequestDate,WaiverGrantedDate,...,AppealStatus,Location,LicensePlateAnon,newlocation,input_string,latitude,longitude,BlockNumber,DT,block
0,0,3001,185322,2016-04-27T11:45:05.000Z,7TH ST NE,11:45,400,VA,NaN,NaN,...,NaN,400 7TH ST NE,106713,"400 7TH ST NE Charlottesville, VA","400 7TH ST NE Charlottesville, VA",38.031652,-78.475925,400,2016-04-27 11:45:05,400 7TH ST NE
1653,1653,3002,185323,2016-04-27T11:46:05.000Z,7TH ST NE,11:46,100,VA,NaN,NaN,...,denied,100 7TH ST NE,114521,"100 7TH ST NE Charlottesville, VA","100 7TH ST NE Charlottesville, VA",38.029756,-78.476534,100,2016-04-27 11:46:05,100 7TH ST NE
2601,2601,3003,185324,2016-04-27T11:49:39.000Z,8TH ST NE,11:49,200,VA,NaN,NaN,...,NaN,200 8TH ST NE,108598,"200 8TH ST NE Charlottesville, VA","200 8TH ST NE Charlottesville, VA",38.029934,-78.476060,200,2016-04-27 11:49:39,200 8TH ST NE
4465,4465,3004,185325,2016-04-27T11:50:59.000Z,PLATEAU RD,11:50,2500,VA,NaN,NaN,...,NaN,2500 PLATEAU RD,125480,"2500 PLATEAU RD Charlottesville, VA","2500 PLATEAU RD Charlottesville, VA",38.024968,-78.520717,2500,2016-04-27 11:50:59,2500 PLATEAU RD
4521,4521,3006,185327,2016-04-27T11:57:08.000Z,5TH ST NE,11:57,200,VA,4/29/16,4/29/16,...,NaN,200 5TH ST NE,39526,"200 5TH ST NE Charlottesville, VA","200 5TH ST NE Charlottesville, VA",38.030625,-78.478093,200,2016-04-27 11:57:08,200 5TH ST NE


In [15]:
b['block'] = b["BlockNumber"].map(str) + " " + b["StreetName"]
b['bI'] = range(b.shape[0])
b.head()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,BlockNumber,StreetName,block,bI
0,400,7TH ST NE,400 7TH ST NE,0
1653,100,7TH ST NE,100 7TH ST NE,1
2601,200,8TH ST NE,200 8TH ST NE,2
4465,2500,PLATEAU RD,2500 PLATEAU RD,3
4521,200,5TH ST NE,200 5TH ST NE,4


In [16]:
b.bI = ['B' + s for s in b.bI.map(str)]
b.head()

/opt/conda/lib/python3.6/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,BlockNumber,StreetName,block,bI
0,400,7TH ST NE,400 7TH ST NE,B0
1653,100,7TH ST NE,100 7TH ST NE,B1
2601,200,8TH ST NE,200 8TH ST NE,B2
4465,2500,PLATEAU RD,2500 PLATEAU RD,B3
4521,200,5TH ST NE,200 5TH ST NE,B4


In [17]:
#unique_ticket_locations_blocks = unique_ticket_locations_blocks.merge(b, how ="left", on="block")

In [18]:
#mat = pd.DataFrame({'month': d.month,'day':d.day})

In [19]:
tickets = tickets.merge(unique_ticket_locations_blocks[['block','latitude','longitude']],how='left',left_on=['latitude','longitude'],right_on=['latitude','longitude'])

In [20]:
tickets = tickets.merge(b[['block','bI']],how='left',on='block')

In [21]:
tickets['DT'] = tickets['DateIssued'].apply(lambda x: datetime.strptime(x, "%Y-%m-%dT%H:%M:%S.%fZ"))

In [22]:
tickets.DT.max(), tickets.DT.min()

(datetime.datetime(5201, 6, 30, 0, 0), datetime.datetime(1999, 1, 5, 0, 0))

In [23]:
tickets.DT[tickets.DT <= datetime(2019,3,1)].max()

datetime.datetime(2018, 7, 26, 14, 2, 37)

In [24]:
tickets.DT[tickets.DT <= datetime(2019,3,1)].max() - pd.Timedelta('3650 days')

datetime.datetime(2008, 7, 28, 14, 2, 37)

In [25]:
d = pd.date_range("07/26/2017","07/26/2018")

In [26]:
index = pd.MultiIndex.from_product([d,t,b.bI], names = ["date","hour","block"])

In [27]:
x = pd.DataFrame(index=index)
x.head()

Empty DataFrame
Columns: []
Index: [(2017-07-26 00:00:00, 00:00:00, B0), (2017-07-26 00:00:00, 00:00:00, B1), (2017-07-26 00:00:00, 00:00:00, B2), (2017-07-26 00:00:00, 00:00:00, B3), (2017-07-26 00:00:00, 00:00:00, B4)]

In [28]:
x.shape

(28073664, 0)

### Format Cville Tickets Data for Merge

In [29]:
tickets['date'] = tickets['DT'].apply(lambda x: datetime.date(x))

In [30]:
tickets['hour'] = tickets['DT'].apply(lambda x: datetime.time(x))

In [31]:
tickets['hour'] = tickets['hour'].map(str)
tickets['hour'] = tickets['hour'].str[:2]

In [32]:
tickets['hour'] = tickets['hour'].map(str) + ":00"

In [33]:
tickets['hour'] = tickets['hour'].apply(lambda x: datetime.strptime(x, "%H:%M").time())

In [34]:
tickets['hour'].head()

0    11:00:00
1    13:00:00
2    13:00:00
3    13:00:00
4    12:00:00
Name: hour, dtype: object

In [35]:
tickets.drop(['block'], axis=1, inplace= True)

In [36]:
tickets.rename(columns={'bI': 'block'}, inplace=True)

In [37]:
tickets.set_index(['date','hour','block'], drop=False, append=False, inplace=True, verify_integrity=False)
tickets.head()

Index  RecordID TicketNumber  \
date       hour     block                                 
2016-04-27 11:00:00 B0         0      3001       185322   
           13:00:00 B0         1      3094       185414   
2016-04-28 13:00:00 B0         2      3136       185462   
                    B0         3      3137       185463   
2016-05-05 12:00:00 B0         4      3614       185970   

                                         DateIssued StreetName TimeIssued  \
date       hour     block                                                   
2016-04-27 11:00:00 B0     2016-04-27T11:45:05.000Z  7TH ST NE      11:45   
           13:00:00 B0     2016-04-27T13:57:39.000Z  7TH ST NE      13:57   
2016-04-28 13:00:00 B0     2016-04-28T13:05:44.000Z  7TH ST NE      13:05   
                    B0     2016-04-28T13:06:43.000Z  7TH ST NE      13:06   
2016-05-05 12:00:00 B0     2016-05-05T12:33:21.000Z  7TH ST NE      12:33   

                          StreetNumber LicenseState WaiverRequestDate  \
date       hour     block                                               
2016-04-27 11:00:00 B0             400           VA               NaN   
           13:00:00 B0             400           VA               NaN   
2016-04-28 13:00:00 B0             400           VA               NaN   
                    B0             400           VA               NaN   
2016-05-05 12:00:00 B0             400           VA               NaN   

                          WaiverGrantedDate    ...          Location  \
date       hour     block                      ...                     
2016-04-27 11:00:00 B0                  NaN    ...     400 7TH ST NE   
           13:00:00 B0                  NaN    ...     400 7TH ST NE   
2016-04-28 13:00:00 B0                  NaN    ...     400 7TH ST NE   
                    B0                  NaN    ...     400 7TH ST NE   
2016-05-05 12:00:00 B0                  NaN    ...     400 7TH ST NE   

                          LicensePlateAnon                        newlocation  \
date       hour     block                                                       
2016-04-27 11:00:00 B0              106713  400 7TH ST NE Charlottesville, VA   
           13:00:00 B0              135871  400 7TH ST NE Charlottesville, VA   
2016-04-28 13:00:00 B0              110072  400 7TH ST NE Charlottesville, VA   
                    B0              132625  400 7TH ST NE Charlottesville, VA   
2016-05-05 12:00:00 B0               71445  400 7TH ST NE Charlottesville, VA   

                                                input_string   latitude  \
date       hour     block                                                 
2016-04-27 11:00:00 B0     400 7TH ST NE Charlottesville, VA  38.031652   
           13:00:00 B0     400 7TH ST NE Charlottesville, VA  38.031652   
2016-04-28 13:00:00 B0     400 7TH ST NE Charlottesville, VA  38.031652   
                    B0     400 7TH ST NE Charlottesville, VA  38.031652   
2016-05-05 12:00:00 B0     400 7TH ST NE Charlottesville, VA  38.031652   

                           longitude block                   DT        date  \
date       hour     block                                                     
2016-04-27 11:00:00 B0    -78.475925    B0  2016-04-27 11:45:05  2016-04-27   
           13:00:00 B0    -78.475925    B0  2016-04-27 13:57:39  2016-04-27   
2016-04-28 13:00:00 B0    -78.475925    B0  2016-04-28 13:05:44  2016-04-28   
                    B0    -78.475925    B0  2016-04-28 13:06:43  2016-04-28   
2016-05-05 12:00:00 B0    -78.475925    B0  2016-05-05 12:33:21  2016-05-05   

                               hour  
date       hour     block            
2016-04-27 11:00:00 B0     11:00:00  
           13:00:00 B0     13:00:00  
2016-04-28 13:00:00 B0     13:00:00  
                    B0     13:00:00  
2016-05-05 12:00:00 B0     12:00:00  

[5 rows x 24 columns]

In [38]:
x.head()

Empty DataFrame
Columns: []
Index: [(2017-07-26 00:00:00, 00:00:00, B0), (2017-07-26 00:00:00, 00:00:00, B1), (2017-07-26 00:00:00, 00:00:00, B2), (2017-07-26 00:00:00, 00:00:00, B3), (2017-07-26 00:00:00, 00:00:00, B4)]

In [39]:
tickets['ticket'] = 1
tickets.head()

Index  RecordID TicketNumber  \
date       hour     block                                 
2016-04-27 11:00:00 B0         0      3001       185322   
           13:00:00 B0         1      3094       185414   
2016-04-28 13:00:00 B0         2      3136       185462   
                    B0         3      3137       185463   
2016-05-05 12:00:00 B0         4      3614       185970   

                                         DateIssued StreetName TimeIssued  \
date       hour     block                                                   
2016-04-27 11:00:00 B0     2016-04-27T11:45:05.000Z  7TH ST NE      11:45   
           13:00:00 B0     2016-04-27T13:57:39.000Z  7TH ST NE      13:57   
2016-04-28 13:00:00 B0     2016-04-28T13:05:44.000Z  7TH ST NE      13:05   
                    B0     2016-04-28T13:06:43.000Z  7TH ST NE      13:06   
2016-05-05 12:00:00 B0     2016-05-05T12:33:21.000Z  7TH ST NE      12:33   

                          StreetNumber LicenseState WaiverRequestDate  \
date       hour     block                                               
2016-04-27 11:00:00 B0             400           VA               NaN   
           13:00:00 B0             400           VA               NaN   
2016-04-28 13:00:00 B0             400           VA               NaN   
                    B0             400           VA               NaN   
2016-05-05 12:00:00 B0             400           VA               NaN   

                          WaiverGrantedDate   ...   LicensePlateAnon  \
date       hour     block                     ...                      
2016-04-27 11:00:00 B0                  NaN   ...             106713   
           13:00:00 B0                  NaN   ...             135871   
2016-04-28 13:00:00 B0                  NaN   ...             110072   
                    B0                  NaN   ...             132625   
2016-05-05 12:00:00 B0                  NaN   ...              71445   

                                                 newlocation  \
date       hour     block                                      
2016-04-27 11:00:00 B0     400 7TH ST NE Charlottesville, VA   
           13:00:00 B0     400 7TH ST NE Charlottesville, VA   
2016-04-28 13:00:00 B0     400 7TH ST NE Charlottesville, VA   
                    B0     400 7TH ST NE Charlottesville, VA   
2016-05-05 12:00:00 B0     400 7TH ST NE Charlottesville, VA   

                                                input_string   latitude  \
date       hour     block                                                 
2016-04-27 11:00:00 B0     400 7TH ST NE Charlottesville, VA  38.031652   
           13:00:00 B0     400 7TH ST NE Charlottesville, VA  38.031652   
2016-04-28 13:00:00 B0     400 7TH ST NE Charlottesville, VA  38.031652   
                    B0     400 7TH ST NE Charlottesville, VA  38.031652   
2016-05-05 12:00:00 B0     400 7TH ST NE Charlottesville, VA  38.031652   

                           longitude  block                   DT        date  \
date       hour     block                                                      
2016-04-27 11:00:00 B0    -78.475925     B0  2016-04-27 11:45:05  2016-04-27   
           13:00:00 B0    -78.475925     B0  2016-04-27 13:57:39  2016-04-27   
2016-04-28 13:00:00 B0    -78.475925     B0  2016-04-28 13:05:44  2016-04-28   
                    B0    -78.475925     B0  2016-04-28 13:06:43  2016-04-28   
2016-05-05 12:00:00 B0    -78.475925     B0  2016-05-05 12:33:21  2016-05-05   

                               hour  ticket  
date       hour     block                    
2016-04-27 11:00:00 B0     11:00:00       1  
           13:00:00 B0     13:00:00       1  
2016-04-28 13:00:00 B0     13:00:00       1  
                    B0     13:00:00       1  
2016-05-05 12:00:00 B0     12:00:00       1  

[5 rows x 25 columns]

In [40]:
tickets.drop_duplicates(subset=['date', 'hour','block'], keep='first', inplace = True)

### Selecting a date range: 7/26/2017 -> 8/26/2017

In [41]:
tickets2 = tickets[tickets.DT >= datetime(2017,7,26)]

In [42]:
#x.to_csv("x.csv")

In [43]:
#tickets2.to_csv("tickets2.csv")

In [44]:
unique_ticket_locations_blocks = unique_ticket_locations_blocks.merge(b[['block','bI']], how = 'left', on = 'block')

### Lat/Long Block Index Lookup Table

In [48]:
blocks = unique_ticket_locations_blocks[['latitude','longitude','block','bI']]

In [52]:
blocks.to_csv("blocklatlong.csv", index= False)

In [51]:
blocks.head()

,latitude,longitude,block,bI
0,38.031652,-78.475925,400 7TH ST NE,B0
1,38.029756,-78.476534,100 7TH ST NE,B1
2,38.029934,-78.476060,200 8TH ST NE,B2
3,38.024968,-78.520717,2500 PLATEAU RD,B3
4,38.030625,-78.478093,200 5TH ST NE,B4


### Merge Model Table and Tickets

In [ ]:
x = x.join(tickets.ticket, how = 'left')

In [ ]:
x.shape
#(28073664, 1)

In [ ]:
x.fillna(value=0, inplace= True)

In [ ]:
x.reset_index(inplace = True)
x['dow'] = x['date'].apply(lambda y: y.weekday())
x['month'] = x['date'].apply(lambda y: y.month)
x['hour'] = x['hour'].str[:2]
x['hour'] = x['hour'].map(int)

In [ ]:
test = x[x.date >= datetime(2018,6,1)]
train = x[x.date < datetime(2018,6,1)]

In [ ]:
test.drop(['date'],inplace= True, axis = 1)

In [ ]:
train = train[['block', 'hour','month','dow','ticket']]
test = test[['block', 'hour','month','dow','ticket']]

In [ ]:
test.ticket.value_counts()
#0.0    4293661
#1.0       1763
# ~2500x

In [ ]:
train.ticket.value_counts()
#0.0    23769374
#1.0        8866

In [ ]:
train2.to_pickle("train2.pkl")
test2.to_pickle("test2.pkl")

# End